<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem III
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Apache Spark;</li>
  <li>Data Wrangling com Spark.</li>
</ol>

---

# **Exercícios**

## 1\. Apache Spark

Replique as atividades do item 2.1 e 2.2 para instalar e configurar um cluster Apache Spark na máquina virtual do Google Colab.

In [1]:
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

In [2]:
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!pip install -q pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 13.8 MB/s eta 0:00:00


In [4]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [5]:
!pip install -q findspark==1.4.2

In [6]:
import findspark

findspark.init()

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("pyspark-notebook").getOrCreate()

## 2\. Data Wrangling

A base de dados presente neste [link](https://www.kaggle.com/datasets/bank-of-england/a-millennium-of-macroeconomic-data) contem dados macroeconômicos sobre o Reino Unido desde o século 13.

**2.1\. Data**

Faça o download dos dados utilizando a máquina virutal do Google Colab com o código abaixo.

In [17]:
!wget -q "https://raw.githubusercontent.com/cluster-apps-on-docker/spark-standalone-cluster-on-docker/master/build/workspace/data/uk-macroeconomic-data.csv" -O "uk-macroeconomic-data.csv"

**2.2. Wrangling**

Processe os dados para que a base de dados final apresente os valores da taxa de desemprego (`Unemployment rate`) e população (`Population (GB+NI)`) estejam ordenados por ano decrescente:

In [19]:
data = spark.read.csv(path="uk-macroeconomic-data.csv", sep=",", header=True)

In [20]:
data.show()

+-----------+------------------------------------+-----------------------------------+-------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------------+----------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------+--------------------+--------------------------+-------------------------------------------------+--------------------+--------------------+---------------------------------------+-------------------------------+---------------------------------+------------------+--------------------+----------+-----------------+---------------------------+------------------------------

Para isso, utilize:

- Pyspark


In [21]:
from pyspark.sql.functions import broadcast

# Selecionar as colunas pedidas
data = data.select(["Description", "Population (GB+NI)", "Unemployment rate"])

# Renomear as mesmas
data = data.\
  withColumnRenamed("Description", 'year').\
  withColumnRenamed("Population (GB+NI)", "population").\
  withColumnRenamed("Unemployment rate", "unemployment_rate")

# Criar um novo df para filtrar a linha com unidades de medida
data_description = data.filter(data['year'] == 'Units')

# Unir ambos os dataframes removendo os valores em comum, no caso a linha com unidades de medida
data = data.join(other=broadcast(data_description), on=['year'], how='left_anti')

# Apagar linhas com dados nulos
data = data.dropna()

# Ordenar dados por ano decrescente
data = data.orderBy(data['year'], ascending=False)

data.show()

+----+----------+-----------------+
|year|population|unemployment_rate|
+----+----------+-----------------+
|2016|     65573|             4.90|
|2015|     65110|             5.38|
|2014|     64597|             6.18|
|2013|     64106|             7.61|
|2012|     63705|             7.97|
|2011|     63285|             8.11|
|2010|     62759|             7.87|
|2009|     62260|             7.61|
|2008|     61824|             5.69|
|2007|     61319|             5.33|
|2006|     60827|             5.42|
|2005|     60413|             4.83|
|2004|     59950|             4.75|
|2003|     59637|             5.01|
|2002|     59366|             5.19|
|2001|     59113|             5.10|
|2000|     58886|             5.46|
|1999|     58684|             5.98|
|1998|     58475|             6.26|
|1997|     58314|             6.97|
+----+----------+-----------------+
only showing top 20 rows



- Pandas

In [13]:
# Converter o dataframe pyspark em dataframe pandas
df = data.toPandas()

# Transformar em csv
df.to_csv('uk-macroeconomic-data.csv', index=False)

# Mostrar dataframe
df.head()

,year,population,unemployment_rate
0,2016,65573,4.90
1,2015,65110,5.38
2,2014,64597,6.18
3,2013,64106,7.61
4,2012,63705,7.97
